In [3]:
import pandas as pd

# Replace 'your_dataset.csv' with the path to your CSV file
#file_path1 = 'analyst_ratings_processed.csv'
file_path1 = 'GoldPrice.csv'

# Load the dataset into a DataFrame
df = pd.read_csv(file_path1)

# Display the first few rows of the DataFrame
print(df.head())


        Dates                                               News
0  13-09-2017          gold suffers third straight daily decline
1  28-02-2018  dent research : is gold's day in the sun comin...
2  06-09-2017  Gold snaps three-day rally as Trump, lawmakers...
3  16-03-2018  april gold holds slight gain, up $2.50, or 0.2...
4  28-07-2017  gold trades in red in early trade; eyes near-t...


In [4]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.nn.functional import softmax

c:\Users\akash\.conda\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\akash\.conda\envs\myenv\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\akash\.conda\envs\myenv\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\akash\.conda\

In [5]:
#!pip install transformers

#Getting the tokenizer and the model

from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading: 100%|██████████| 758/758 [00:00<?, ?B/s] 
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 468kB/s] 
Downloading: 100%|██████████| 112/112 [00:00<?, ?B/s] 
Downloading: 100%|██████████| 418M/418M [00:57<00:00, 7.58MB/s] 


In [6]:
# Function to perform sentiment analysis
def analyze_sentiment(news_text):
    tokens = tokenizer.encode_plus(news_text, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    input_ids = tokens["input_ids"]
    attention_mask = tokens["attention_mask"]

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        probs = softmax(outputs.logits, dim=-1)  # Get probabilities
        label = torch.argmax(probs, dim=1).item()  # Get label (0, 1, or 2)
    
    # Map labels to sentiments
    label_map = {0: "negative", 1: "neutral", 2: "positive"}
    return label_map[label], probs.numpy()[0]



In [ ]:
# Apply sentiment analysis on the 'news' column
df["sentiment"], df["probabilities"] = zip(*df["News"].apply(analyze_sentiment))

# Display the resulting DataFrame
print(df)


In [ ]:
df.to_csv("gold.csv", index=False)